<a href="https://colab.research.google.com/github/dhinu95/Get-latitude-and-longitude-from-address/blob/main/Lat%26Lng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#import python libraries
import pandas as pd
import requests
import logging
import time

In [10]:
#upload input & output CSV file.
from google.colab import files
data_to_load = files.upload()

Saving output.csv to output.csv


In [11]:
# create logger - log messages that you want to see.
logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)

# create console handler.
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

In [12]:
#get google geocode API key from 'https://developers.google.com/maps/documentation/geocoding/get-api-key'.
API_KEY = None
BACKOFF_TIME = 30

#set output file.
output_filename = 'output.csv'

#set input file.
input_filename = 'address.csv'

#Specify the column name in your input data that contains addresses here.
address_column_name = "Address"

#If True, full JSON results from Google are included in output.
RETURN_FULL_RESULTS = False

In [13]:
#read the data
data = pd.read_csv(input_filename, encoding='utf8')

In [14]:
#if no address column return missing message.
if address_column_name not in data.columns:
	raise ValueError("Missing Address column in input data")
 
#Form a list of addresses for geocoding.
addresses = data[address_column_name].tolist()

In [15]:
def get_google_results(address, api_key=None, return_full_response=False):
    
     # Set up your Geocoding url.
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
    if api_key is not None:
        geocode_url = geocode_url + "&key={}".format(api_key)
        
    #Ping google for the reuslts.
    results = requests.get(geocode_url)
    #Results will be in JSON format - convert to dict using requests functionality.
    results = results.json()
    
    # if there's no results or an error, return empty results.
    if len(results['results']) == 0:
        output = {
            "formatted_address" : None,
            "latitude": None,
            "longitude": None,
        }
    else:    
        answer = results['results'][0]
        output = {
            "formatted_address" : answer.get('formatted_address'),
            "latitude": answer.get('geometry').get('location').get('lat'),
            "longitude": answer.get('geometry').get('location').get('lng')
        }
        
    #Append some other details.    
    output['input_string'] = address
    output['number_of_results'] = len(results['results'])
    output['status'] = results.get('status')
    if return_full_response is True:
        output['response'] = results
    return output


In [16]:
# before we start, that the API key is ok/valid, and internet access is ok.
test_result = get_google_results('India', API_KEY, RETURN_FULL_RESULTS)
if (test_result['status'] != 'OK') or (test_result['formatted_address'] != 'India'):
    logger.warning("There was an error when testing the Google Geocoder.")
    raise ConnectionError('Problem with test results from Google Geocode - check your API key and internet connection.')

In [ ]:
# Create a list to hold results.
results = []

# Go through each address in turn.
for address in addresses:
    # While the address geocoding is not finished.
    geocoded = False
    while geocoded is not True:
        # Geocode the address with google
        try:
            geocode_result = get_google_results(address, API_KEY, return_full_response=RETURN_FULL_RESULTS)
        except Exception as e:
            logger.exception(e)
            logger.error("Major error with {}".format(address))
            logger.error("Skipping!")
            geocoded = True
            
        # If we're over the API limit, backoff for a while and try again later.
        if geocode_result['status'] == 'OVER_QUERY_LIMIT':
            logger.info("Hit Query Limit! Backing off for a bit.")
            time.sleep(BACKOFF_TIME * 60) # sleep for 30 minutes
            geocoded = False
        else:
            # If we're ok with API use, save the results
            # Note that the results might be empty / non-ok - log this
            if geocode_result['status'] != 'OK':
              logger.warning("Error geocoding {}: {}".format(address, geocode_result['status']))
            logger.debug("Geocoded: {}: {}".format(address, geocode_result['status']))
            results.append(geocode_result)           
            geocoded = True

In [ ]:
# print meassage using logger
logger.info("Finished geocoding of all input addresses")
# Write the full results to csv using the pandas library.
pd.DataFrame(results).to_csv(output_filename, encoding='utf8')
